In [1]:
import tensorflow as tf
import numpy as np
import scipy.io
from pyDOE import lhs
import math

D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1ty

In [2]:
RandomSeed = 1235
np.random.seed(RandomSeed)
tf.set_random_seed(RandomSeed)

In [3]:
class PtPINNsss:
    # Initialize the class
    def __init__(self, x, t, u, lb, ub, lbp,ubp, layers,weights_values,biases_values):
        
        X = np.concatenate([x, t], 1)

        self.X = X
        
        self.x = X[:,0:1]
        self.t = X[:,1:2]
        
        self.u = u      
        self.lb = lb
        self.ub = ub
        self.hsadasjd=0
        self.ubp = ubp
        self.lbp = lbp        
        # Initialize NNs
        self.layers = layers
        self.hh=[]
        self.hh1=[]      
        
        self.weights, self.biases = self.initialize_NN(layers)
        
        value_float64 = 3.0/4
        self.b = tf.Variable(value_float64, dtype=tf.float64)
        
        value_float64=  1.0/(math.cosh(5*0.5)-1)       
        
        self.a = tf.constant(value_float64, dtype=tf.float64)   
     
        
    
        
        self.weights_values = weights_values        
        self.biases_values = biases_values    

        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))    

        self.x_f_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        
        self.x_lb_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_b_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        self.x_ub_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        
        self.x_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        
        self.u_tf = tf.placeholder(tf.float64, shape=[None, self.u.shape[1]])

        self.u_pred, _ ,self.uuuuuu = self.net_AC(self.x_tf, self.t_tf)
        self.u_lb_pred, self.ux_lb_pred,_ = self.net_AC(self.x_lb_tf, self.t_b_tf)
        self.u_ub_pred, self.ux_ub_pred,_ = self.net_AC(self.x_ub_tf, self.t_b_tf)

        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)
        


        
        self.lossS = tf.reduce_mean(tf.square(self.u_tf - self.u_pred))                                              
        self.lossB = tf.reduce_mean(tf.square(self.u_lb_pred - self.u_ub_pred)) + tf.reduce_mean(tf.square(self.ux_lb_pred - self.ux_ub_pred))
        self.lossfu =  tf.reduce_mean(tf.square(self.f_pred))    
        

        self.optimizer_Adam = tf.train.AdamOptimizer()

        self.loss  =   self.lossS + 20 *   self.lossB + self.lossfu
             
        
        
        
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)      

        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
    
        init = tf.global_variables_initializer()
        self.sess.run(init)
        self.save = tf.train.Saver(max_to_keep=1)
              
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = tf.Variable(tf.zeros([layers[l], layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)        
        return weights, biases
    
    
    def initialize_NN2(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)        
        return weights, biases      
    
    
    
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev, dtype=tf.float64))


    def custom_function(self,t):
        
        ab= self.b     
        
        condition2 = tf.less_equal(t, 0.5)
        condition3 = tf.less_equal(t, ab)
        condition4 = tf.less_equal(ab, 1)
        
        BB=2*(t-0.5)
        
        
        B=1/(ab-0.5)*(t-0.5)
        
        return tf.where(condition2, tf.zeros_like(t), tf.where(condition4,tf.where(condition3, -2*B**3+3*B**2,tf.zeros_like(t)+1),-2*BB**3+3*BB**2))
    
    def neural_net(self, x,t, weights, biases,weights_values,biases_values):

        tt=2*t
        x = (x+1)/2
        num_layers = len(weights) + 1
        H = tf.concat([tt,x],1)
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            W1 = weights_values[l]
            b1 = biases_values[l]   
            H0 = tf.add(tf.matmul(H, W), b)
            H1 = tf.add(tf.matmul(H, W1), b1)
            
            B = self.custom_function(t)   
            H0 = H0*B              
            
            H  = tf.add(H0,H1)
            H =  tf.tanh(H)
        
        W = weights[-1]
        b = biases[-1]
        W1 = weights_values[-1]
        b1 = biases_values[-1]  
        H0 = tf.add(tf.matmul(H, W), b)
        
        B = self.custom_function(t)     
        H0 = H0*B            
        
        H1 = tf.add(tf.matmul(H, W1), b1) 
        H  = tf.add(H0,H1)      
        Y = H
        return Y
    
    def net_AC(self, x, t):
        u = self.neural_net(x,t, self.weights, self.biases,self.weights_values, self.biases_values)
        u_x = tf.gradients(u, x)[0]
        u_t = tf.gradients(u, t)[0]

        return u, u_x, u_t
    
       
    

    def net_f(self, x, t):
        u, u_x, u_t = self.net_AC(x, t)
        u_xx = tf.gradients(u_x, x)[0]
        u_xxx = tf.gradients(u_xx, x)[0]            
        f_u = u_t +  u*u_x + 0.0025*u_xxx
        return f_u
    
    def callback(self, loss, a,b,lossfu, lossS, lossB):
        sss=self.hsadasjd
        if sss%1000==0:
            print('Loss: %.6e, Lossfu: %.3e, LossS: %.3e, LossB: %.3e ' % (loss, lossfu, lossS, lossB))
        sss=sss+1
        self.hsadasjd=sss 
        self.hh.append(a)      
        self.hh1.append(b)        
    def train(self, nIter, Nf, Nn, Nb):

        X_train = self.lbp + (self.ubp-self.lbp)*lhs(2, Nf)
        self.xtrain_f = X_train[:,0:1]
        self.ttrain_f = X_train[:,1:2] 
        
        X_lb_train = self.lbp + [0,self.ubp[1]-self.lbp[1]]*lhs(2, Nb)
        self.xtrain_lb = X_lb_train[:,0:1]
        self.ttrain_b = X_lb_train[:,1:2]
        
        X_ub_train = [self.ubp[0],self.lbp[1]] + [0,self.ubp[1]-self.lbp[1]]*lhs(2, Nb)
        self.xtrain_ub = X_ub_train[:,0:1]
        
        tf_dict = {self.x_tf: self.x, self.t_tf: self.t, self.u_tf: self.u,
                   self.x_lb_tf: self.xtrain_lb, self.t_b_tf: self.ttrain_b, 
                   self.x_ub_tf: self.xtrain_ub, 
                   self.x_f_tf: self.xtrain_f, self.t_f_tf: self.ttrain_f}

        for it in range(nIter):
            loss_value = self.sess.run(self.loss, tf_dict)
            lossfu = self.sess.run(self.lossfu, tf_dict)
            lossS = self.sess.run(self.lossS, tf_dict)
            lossB = self.sess.run(self.lossB, tf_dict)
            
            self.sess.run(self.train_op_Adam, tf_dict)
                
            ab= self.sess.run(self.a)
            self.hh.append(ab)       
            abc= self.sess.run(self.b)
            self.hh1.append(abc)    

            print(abc)                 
            if it % 1000 == 0:
                print('It: %d, Loss: %.6e, Lossfu: %.3e, LossS: %.3e, LossB: %.3e' % (it, loss_value, lossfu, lossS, lossB))
                

        
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, method = 'L-BFGS-B', options = {'maxiter': 50000,'maxfun': 50000,'maxcor': 50,'maxls': 50,'ftol' : 1.0 * np.finfo(float).eps})                                                                                                         
        self.optimizer.minimize(self.sess, feed_dict = tf_dict, fetches = [self.loss,self.a,self.b,  self.lossfu, self.lossS, self.lossB], loss_callback = self.callback)        
                                    
    
    def predict(self, x, t):
        
        tf_dict = {self.x_tf: x, self.t_tf: t}
        u_star = self.sess.run(self.u_pred, tf_dict)
        u_starssss = self.sess.run(self.uuuuuu, tf_dict)        
        return u_star,u_starssss

    def saver(self, string):
        self.save.save(self.sess, 'ckpt/'+string)
        
        
    def sssss(self):
        return self.hh,  self.hh1           
        
        
    def restore(self):
        model_file = tf.train.latest_checkpoint('ckpt/')
        self.save.restore(self.sess, model_file)

In [4]:
import pickle

In [5]:
with open('bcweights.pkl', 'rb') as f:
    weights_values = pickle.load(f)

In [6]:
with open('bcweights1.pkl', 'rb') as f:
    biases_values = pickle.load(f)

In [7]:
if __name__ == "__main__": 
           
    
    # Doman bounds
    lb = np.array([-1, 0])
    ub = np.array([1, 0.5])
    
    lb1 = np.array([-1, 0.5])
    ub1 = np.array([1, 1.0]) 
    
    layers = [2,30,30,30,1]
    
    data = scipy.io.loadmat('kdv.mat')
    
    t = data['t'].flatten()[:,None]
    x = data['x'].flatten()[:,None]
    Exact = data['usol']
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]

    def IC(x):
        u = np.cos(np.pi*x)
        return u

    N0 = 400
    x=np.linspace(-1,1,N0).flatten()[:,None]  
    X0 =x
    T0 = np.full((N0,1), lb[1])
    U0 = IC(X0)
      
    model2 = PtPINNsss(X0, T0, U0, lb, ub,lb1, ub1, layers,weights_values,biases_values)                        
    model2.train(5000 ,4000, 400,400)  
    model2.saver('testmodel.ckpt')
    


    u_pred ,_= model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))

    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
0.75
It: 0, Loss: 1.375065e+01, Lossfu: 7.825e+00, LossS: 3.562e-09, LossB: 2.963e-01
0.7507441342747971
0.7516026139313098
0.7525137844455182
0.7534553157501882
0.7544200754170113
0.7554025961158607
0.7563963578945465
0.7574028641274155
0.7584219770760968
0.7594491137533065
0.7604767273095167
0.7615006718588754
0.7625202174780121
0.7635358612284363
0.7645525335086136
0.7655759022353291
0.7666038596297472
0.7676267268733722
0.768632002815832
0.7696109257743946
0.7705604952790555
0.7714828649735123
0.7723842743730736
0.7732698665706311
0.7741377060045709
0.7749792473195065
0.7757854819116068
0.7765516648558612
0.7772782661134362
0.777969891771821
0.7786336662272577
0.7792747349203013
0.7798933229157375
0.7804850539674046
0.7810469949510868
0.7815789292471529
0.7820841031072736
0.7825669074352454
0.7830318954007056
0.7834808766358865
0.7839122320437185
0.784323273111790

0.8416162538124454
0.8417071923561765
0.8417978763846853
0.841888318298218
0.8419785262253815
0.842068529499029
0.8421582585001588
0.842247727884658
0.8423369506249383
0.8424259559676159
0.8425147530231121
0.8426033501543982
0.8426917547699754
0.8427799709975643
0.842868003995074
0.842955920680228
0.8430437238995672
0.8431314127578341
0.8432190245191641
0.8433065546039675
0.8433939987034212
0.8434813531451015
0.8435685922413572
0.8436557191795138
0.8437427328621964
0.8438296421458032
0.8439164451470912
0.8440031655403102
0.8440897993108043
0.8441763426644356
0.8442627975229989
0.8443491602121036
0.8444354250327623
0.8445215891425393
0.8446076501356766
0.8446936211044821
0.8447794980486704
0.8448652779871836
0.8449509582293993
0.8450365506060503
0.8451220586532491
0.8452074783001405
0.8452928058194876
0.8453780379861464
0.8454631715895102
0.8455482040555079
0.845633113313942
0.8457179208513587
0.8458026235446097
0.8458872194762412
0.8459717130413412
0.8460560887475204
0.8461403592903797

0.8692721344102308
0.869313892753926
0.8693555102489369
0.8693971435644214
0.8694386194029982
0.8694801954139824
0.8695215349317481
0.869563216665065
0.8696046065246381
0.8696470571078669
0.8696894439012616
0.8697352905382216
0.869783615621578
0.8698446701991753
0.8699202353998106
0.8700375365710875
0.8701943989742443
0.8704120113958878
0.8706390409186138
0.8708520035147886
0.8710524643221936
0.871255330642727
0.8714794446817691
0.8716914382125843
0.8718861008649295
0.8720844425652446
0.8722788796041723
0.8724653078082338
0.8726389241644672
0.8728043052339075
0.8729718270576332
0.8731271754152311
0.8732713719573587
0.8734167021330608
0.8735527928258727
0.8736791917725503
0.8738019168437844
0.8739168975632157
0.8740262192159729
0.8741281513067728
0.874223433597794
0.87431847964034
0.8744058459499622
0.8744867244997738
0.8745681137456842
0.8746436345818317
0.8747145102279136
0.8747823861637731
0.8748456367140206
0.8749082471946034
0.874966385606271
0.8750201906285501
0.8750745073614654
0

0.8854121361551494
0.8854096744518898
0.8854072714021779
0.8854049473669077
0.8854026813756595
0.885400476973696
0.8853983285458185
0.8853962445457242
0.8853942295548723
0.8853922646987752
0.8853903642688542
0.8853885247666148
0.8853867462347093
0.8853850262636943
0.8853833614368861
0.8853817628636703
0.885380217181363
0.8853787272075009
0.8853772964662654
0.8853759259551381
0.8853746174192003
0.8853733623547463
0.8853721690487787
0.8853710334583194
0.8853699530057959
0.8853689294070952
0.8853679619310517
0.8853670544410951
0.8853661993568198
0.8853654000847211
0.885364658080161
0.8853639715055845
0.8853633403352879
0.8853627615464494
0.8853622389886162
0.8853617683012462
0.88536134932601
0.8853609835430464
0.8853606702131291
0.8853604092551701
0.8853601966854702
0.8853600350753363
0.8853599227658481
0.8853598585260473
0.8853598421049566
0.8853598728741889
0.8853599516404146
0.8853600752588713
0.8853602450734597
0.8853604606808342
0.8853607213985624
0.8853610259816426
0.885361373326231

0.8897783150666255
0.8897807761076507
0.8897873754651136
0.8897997913400648
0.8898250925112425
0.8898694861801859
0.8899519221736193
0.8900846950349864
0.8902989447105778
0.8905805940860329
0.8909161973621503
0.8912460283983354
0.8915423419516575
0.8918180096159715
0.8920986449892756
0.8923980941848311
0.8926836395261886
0.8929402486824911
0.8931803972868704
0.8934151596007821
0.8936514714543218
0.8938652417001952
0.8940558221337778
0.8942408853245178
0.8944197850871416
0.8945905407327974
0.894741689506536
0.894876096972916
0.8950080014389893
0.8951281178424685
0.8952365512709309
0.8953327367939938
0.8954173818216317
0.8955009338803716
0.8955734851612318
0.8956367625636126
0.895694175274153
0.8957431369493406
0.8957901058972636
0.8958282428759535
0.8958594863720567
0.8958879176691851
0.8959110582047337
0.8959330131726263
0.8959490398671228
0.8959601250735694
0.8959692070279732
0.8959743896407389
0.8959781467180318
0.8959774580395
0.8959731605996362
0.895967212250114
0.8959589774802711


...

0.9080648697550484
0.9080437199591911
0.9080228571380484
0.9080023370633389
0.9079820906335135
0.9079621425852739
0.9079424342427933
0.9079230063267781
0.9079038131426794
0.9078848822704984
0.9078661805419191
0.9078477242002608
0.9078294945909254
0.9078114978548762
0.9077937240498283
0.907776163579989
0.9077588147797782
0.9077416648305926
0.9077247177651705
0.907707955877112
0.9076913924994912
0.907675006494971
0.9076588146407135
0.9076427902112957
0.907626956031143
0.9076112802636188
0.9075957920054896
0.9075804490379514
0.9075652901598129
0.9075502697062062
0.907535435660747
0.9075207296129602
0.9075062129542981
0.9074918095352388
0.9074775998810624
0.9074634846416157
0.907449574727777
0.9074357423279698
0.9074221425129503
0.9074085883875713
0.9073952997570486
0.907382012927307
0.9073690455121639
0.9073560182709108
0.9073434003079073
0.9073306350930314
0.907318429215191
0.9073059548463741
0.9072943019570504
0.907282230111863
0.9072714617759288
0.9072601267318904
0.9072510392232893
0.

0.9204015126536368
0.9206439144763093
0.9208329267437588
0.9210134235232031
0.9211388015608527
0.9212189811704495
0.9212883000730198
0.9213194863219832
0.9213543890984832
0.9213551284484628
0.9213311660873171
0.9212954843589375
0.9212346632214695
0.9211777878674697
0.9210975442237215
0.9210053071983619
0.920899412763959
0.9207758645253652
0.9206545108672002
0.9205188341328311
0.920383560805159
0.9202410058204613
0.9200931072820131
0.9199504151730381
0.9198023793828214
0.9196600927373324
0.9195122694404636
0.9193623079510554
0.9192117069426544
0.9190560066797835
0.9189049605587312
0.9187498691248855
0.9185981105255914
0.9184457814582939
0.9182937435136641
0.9181471396692839
0.9180004917022256
0.9178609451517441
0.9177224680099242
0.9175878971181085
0.9174561115098055
0.917324936199171
0.9171980335844413
0.9170708812818494
0.9169477386392277
0.9168256580239643
0.9167066153614688
0.9165907101953913
0.9164765983689512
0.9163670276333266
0.9162588054973263
0.91615487828985
0.916052589696901

0.9185534434439837
0.9184988800200208
0.9184472647801082
0.9183968246652662
0.918349047546408
0.9183026785211574
0.918258405035173
0.9182157157114185
0.9181745607748377
0.9181352639909587
0.9180970127403196
0.9180607895881648
0.9180252718858699
0.9179915739824684
0.9179583712071924
0.9179264655639311
0.9178950274831962
0.917864578540146
0.9178347246862945
0.9178056289296899
0.9177772760707261
0.917749464022466
0.9177224850075444
0.9176958158290863
0.9176700633763424
0.9176445335741242
0.9176199049254103
0.9175954803615182
0.9175718418684874
0.9175483974546212
0.9175255935816458
0.9175029901604004
0.9174809473959168
0.9174591591597823
0.9174378800673793
0.9174169530568137
0.9173964510174956
0.9173763678263385
0.9173565114070789
0.9173369920897054
0.9173175620440952
0.9172984066677503
0.9172793251003003
0.917260546459486
0.9172418126479018
0.9172234223712519
0.9172050587668378
0.9171871127586895
0.917169218778341
0.9171517806723682
0.9171343778074789
0.9171174439586081
0.9171004958508776

0.9248905806650185
0.9248127751816947
0.924743015611957
0.924675414151257
0.9246130440982153
0.9245548858251363
0.9245000523488712
0.9244498336141356
0.9244020233670592
0.92435835945196
0.9243156480588061
0.9242769189527854
0.9242382360062539
0.9242035161356967
0.9241695499977183
0.9241386360840644
0.924110168018348
0.9240829630163585
0.9240593967835508
0.9240355476418438
0.9240151002315119
0.9239941500675852
0.9239753719430244
0.9239568523110435
0.9239390505174452
0.9239221104722789
0.923905151432347
0.9238892131250644
0.9238732581958525
0.9238582769021808
0.9238432049764831
0.9238291741703721
0.9238147267195149
0.9238012883846434
0.9237873159903867
0.9237741912959504
0.9237608147738365
0.9237478539471349
0.923735198774881
0.9237224247692979
0.9237103631692464
0.9236978933258932
0.9236862171287776
0.9236741782854346
0.9236627688014789
0.9236511863913351
0.923640072919926
0.923628836421604
0.9236179895584996
0.9236069312185118
0.9235962757513979
0.9235856606675847
0.9235755706189361
0.

Loss: 2.706530e-04, Lossfu: 2.609e-04, LossS: 3.562e-09, LossB: 4.861e-07 
Loss: 2.628669e-04, Lossfu: 2.531e-04, LossS: 3.562e-09, LossB: 4.860e-07 
Loss: 2.547296e-04, Lossfu: 2.450e-04, LossS: 3.562e-09, LossB: 4.847e-07 
Loss: 2.483511e-04, Lossfu: 2.393e-04, LossS: 3.562e-09, LossB: 4.519e-07 
Loss: 2.409690e-04, Lossfu: 2.336e-04, LossS: 3.562e-09, LossB: 3.688e-07 
Loss: 2.359978e-04, Lossfu: 2.285e-04, LossS: 3.562e-09, LossB: 3.732e-07 
Loss: 2.306223e-04, Lossfu: 2.232e-04, LossS: 3.562e-09, LossB: 3.725e-07 
Loss: 2.259612e-04, Lossfu: 2.180e-04, LossS: 3.562e-09, LossB: 3.974e-07 
Loss: 2.221669e-04, Lossfu: 2.146e-04, LossS: 3.562e-09, LossB: 3.801e-07 
Loss: 2.187164e-04, Lossfu: 2.107e-04, LossS: 3.562e-09, LossB: 3.992e-07 
Loss: 2.153936e-04, Lossfu: 2.077e-04, LossS: 3.562e-09, LossB: 3.845e-07 
Loss: 2.110761e-04, Lossfu: 2.036e-04, LossS: 3.562e-09, LossB: 3.739e-07 
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of f AND g EVALUATIONS EX

In [8]:
    t = data['t'].flatten()[:,None][0:101] 
    x = data['x'].flatten()[:,None]
    Exact = data['usol'][:,0:101]
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred ,f_pred= model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))
    U_pred=u_pred.reshape(101,512).T
    U_preds=U_pred

randorm seed: 1235
Error2 u: 9.171897e-04
Error1 u: 4.717081e-04
Errorf u: 3.023703e-03


In [9]:
    t = data['t'].flatten()[:,None]
    x = data['x'].flatten()[:,None]
    Exact = data['usol']
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred ,f_pred= model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))
    U_pred=u_pred.reshape(201,512).T
    U_preds=U_pred

randorm seed: 1235
Error2 u: 3.903844e-03
Error1 u: 1.477899e-03
Errorf u: 1.737560e-02


In [10]:
    scipy.io.savemat("aednn.mat", {'u': u_pred})

In [11]:
    scipy.io.savemat("aedf.mat", {'u': f_pred})

In [12]:
    xielv,sss=model2.sssss()

In [13]:
    scipy.io.savemat("aegpinna.mat", {'egpinna': xielv})
    scipy.io.savemat("aegpinnb.mat", {'egpinnb': sss})